In [36]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  #
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import seaborn as sns
import statsmodels.api as sm
import warnings
#warnings.filterwarnings("ignore")

# Trading Strategy Analysis

This notebook provides an analysis of a trading strategy with respect to market data. It is divided into three main sections:

1. **Library Installation**: Libraries required for analysis.
2. **Data Cleanup and Preprocessing**: Steps taken to clean the data and prepare it for analysis.
3. **Trading Strategy and Final Summary**: Implementing a trading strategy, calculating performance, and providing a summary of the results.


### 1. Library Installation

We need to install the following libraries to perform the analysis:

- **pandas** for data manipulation.
- **numpy** for numerical operations.
- **matplotlib** for plotting.
- **sklearn** for standard scaling.
- **seaborn** for graph.
- **statsmodels** for stats
- **openpyxl** for excel handling.


## 2. Data Cleanup and Preprocessing

In this section, the following data cleanup steps were performed to prepare the dataset for further analysis:

1. **Load Data**: We loaded the data from a given file path and sheet name.
2. **Remove Empty Columns**: Any columns that is completely empty are removed.
3. **Convert Date Column**: The date column was converted into a datetime format and set as the index.
4. **Fill Missing Values**: Missing values in the data were filled using appropriate methods.
5. **Remove Outliers and Replace with Rolling Median**: Outliers in the 'Economic Index' were removed and replaced with rolling medians.
6. **Fill Missing Monthly Data**: Missing data for certain months was filled in a structured manner (e.g., using interpolation or forward filling) without lossing the trend in the data.
7. **Normalize Yearly Data**: Data for selected columns were normalized yearly (commented out in the provided code).


In [33]:
import pandas as pd

def load_data(file_path, sheet_name):
    data = pd.read_excel(file_path, sheet_name=sheet_name)
    return data

def remove_empty_columns(data):
    data = data.dropna(axis=1, how='all')
    return data

def convert_date_column(data, date_column, setAsIndex = False):
    data[date_column] = pd.to_datetime(data[date_column], errors='coerce')
    invalid_dates = data[data[date_column].isna()]
    print(f"Rows with invalid dates:\n{invalid_dates}")    
    if setAsIndex:
        data.set_index(date_column, inplace=True)    
    return data

def fillNA(data):
    data['Stock Market'] = data['Stock Market'].interpolate(method='linear')
    data['RSI'] = data['RSI'].interpolate(method='linear')
    data['RSI'] = data['RSI'].interpolate(method='linear')
    data['Forward EPS Estimate'] = data['Forward EPS Estimate'].interpolate(method='linear')
    data['Commodities Price Index'] = data['Commodities Price Index'].interpolate(method='linear')
    data['Economic Index'] = data['Economic Index'].interpolate(method='linear')
    return data
    
def removeOutlierAndReplaceWithRollingMedian(data, column_name, lower_bound=-100, upper_bound=100, window_size=25):
    data['Outlier Flag'] = (data[column_name] < lower_bound) | (data[column_name] > upper_bound)
    data['Rolling Median'] = data[column_name].rolling(window=window_size, min_periods=1).median()
    data[column_name] = data.apply(
        lambda row: row['Rolling Median'] if row['Outlier Flag'] else row[column_name],
        axis=1
    )
    data.at["2023-07-15", column_name] = data.at['2023-07-15', 'Rolling Median']
    data = data.drop(columns=['Rolling Median', 'Outlier Flag'])
    return data

def fillMonthlydata(data):
    data_month_start = data['Average Precipitation over last month'].resample('MS').first()
    data['Average Precipitation over last month'] = data['Average Precipitation over last month'].fillna(data_month_start)
    data['Average Precipitation over last month'] = data['Average Precipitation over last month'].ffill()
    return data

import pandas as pd

def normalize_yearly(data, columns):
    data_yearly = data.resample('YS').agg({col: ['mean', 'std'] for col in columns})
    
    # Normalize for each column
    for col in columns:
        data[col + '_Normalized'] = data[col] - data[col].resample('YS').transform('mean')
        data[col + '_Normalized'] /= data[col].resample('YS').transform('std')
    
    return data    
    
def clean_data(file_path, sheet_name, date_column='Date'):
    columns_to_normalize = ['Stock Market','Forward EPS Estimate', 'Economic Index', 'Commodities Price Index']
    data = load_data(file_path, sheet_name)
    data = remove_empty_columns(data)
    data = convert_date_column(data, date_column, setAsIndex = True)
    data = fillNA(data)
    data = removeOutlierAndReplaceWithRollingMedian(data, 'Economic Index')
    data = fillMonthlydata(data)
    #data = normalize_yearly(data, columns_to_normalize)
    return data

def plot_correlation_heatmap(data):
    # Calculate the correlation matrix
    corr_matrix = data.corr()

    # Plot the heatmap
    plt.figure(figsize=(10, 8))  # Adjust the figure size as needed
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5, cbar_kws={'shrink': 0.8})

    # Set plot title and labels
    plt.title('Correlation Matrix Heatmap')
    plt.show()

# Example usage
file_path = 'Data Intel Project - Data Only.xlsx'
sheet_name = 'Returns Static'
data = clean_data(file_path, sheet_name)

Rows with invalid dates:
Empty DataFrame
Columns: [Date, Stock Market, RSI, News Sentiment, Forward EPS Estimate, Economic Index, Commodities Price Index, Average Precipitation over last month]
Index: []


In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class TradingStrategy:
    def __init__(self, data):
        # Drop null values from data
        self.data = data.dropna(subset=['RSI', 'Forward EPS Estimate', 'Stock Market'])
        self.data.reset_index(drop=False, inplace=True)  # Reset the index to make 'Date' a column
        self.signals = pd.DataFrame(index=self.data.index)
        
        # Ensure 'Date' is a datetime object (if it's not already)
        self.data['Date'] = pd.to_datetime(self.data['Date'])

    def calculate_indicators(self):
        # Bollinger Bands
        rolling_20 = self.data['Stock Market'].rolling(window=20)
        self.data['20_MA'] = rolling_20.mean()
        self.data['Upper_Band'] = self.data['20_MA'] + 2 * rolling_20.std()
        self.data['Lower_Band'] = self.data['20_MA'] - 2 * rolling_20.std()

        # RSI Calculation
        delta = self.data['Stock Market'].diff(1)
        gain = np.where(delta > 0, delta, 0)
        loss = np.where(delta < 0, -delta, 0)
        avg_gain = pd.Series(gain).rolling(window=14).mean()
        avg_loss = pd.Series(loss).rolling(window=14).mean()
        self.data['RSI'] = 100 - (100 / (1 + avg_gain / avg_loss))

        # Forward EPS Median
        #self.data['EPS_Median'] = self.data['Forward EPS Estimate'].rolling(window=20).median()

    def generate_signals(self):
        # Extract month directly from the index (datetime)
        self.data['Month'] = self.data['Date'].dt.month
        
        # Buy Signals - Only in May
        self.signals['Buy'] = (
            (self.data['Month'] >= 2) & 
            (self.data['Month'] <= 4) 
            #(self.data['Stock Market'] < self.data['Lower_Band']) &
            #(self.data['RSI'] > 20) &
            #(self.data['RSI'] < 35)
        )
        
        # Sell Signals - Only in November
        self.signals['Sell'] = (
            (self.data['Month'] >= 9) & 
            (self.data['Month'] <= 10)
            #(self.data['Stock Market'] > self.data['Upper_Band']) &
            #(self.data['RSI'] > 90)
        )

    def calculate_profit(self):
        # Simple Profit Calculation
        self.data['Position'] = 0
        self.data.loc[self.signals['Buy'], 'Position'] = 1  # Buy Position in May
        self.data.loc[self.signals['Sell'], 'Position'] = 0  # Sell Position in November
        self.data['Position'] = self.data['Position'].ffill()  # Maintain position until next action
        self.data['Daily_Return'] = self.data['Stock Market'].pct_change()
        self.data['Strategy_Return'] = self.data['Daily_Return'] * self.data['Position']
        self.data['Cumulative_Return'] = (1 + self.data['Strategy_Return']).cumprod()
    
        # Track profit for each buy-sell pair
        profit_details = []
    
        for buy_date, sell_date in zip(self.data[self.signals['Buy']].index, self.data[self.signals['Sell']].index):
            buy_price = self.data.loc[buy_date, 'Stock Market']
            sell_price = self.data.loc[sell_date, 'Stock Market']
            buy_rsi = self.data.loc[buy_date, 'RSI']
            sell_rsi = self.data.loc[sell_date, 'RSI']
            buy_eps = self.data.loc[buy_date, 'Forward EPS Estimate']
            sell_eps = self.data.loc[sell_date, 'Forward EPS Estimate']
    
            # Calculate profit from buy to sell
            profit = sell_price - buy_price
    
            # Append details of the trade to profit_details list
            profit_details.append({
                'Buy Price': buy_price,
                'Sell Price': sell_price,
                'Buy RSI': buy_rsi,
                'Sell RSI': sell_rsi,
                'Buy EPS': buy_eps,
                'Sell EPS': sell_eps,
                'Profit': profit
            })
    
        # Convert profit details to DataFrame
        profit_df = pd.DataFrame(profit_details)
    
        # Calculate total profit from all trades
        total_profit = profit_df['Profit'].sum()
    
        # Add a row for the total profit
        total_row = pd.DataFrame([{
            'Buy Price': '',
            'Sell Price': '',
            'Buy RSI': '',
            'Sell RSI': '',
            'Buy EPS': '',
            'Sell EPS': '',
            'Profit': total_profit
        }])
    
        # Append total row to the profit_df
        profit_df = pd.concat([profit_df, total_row], ignore_index=True)
    
        # Print the profit table
        print(profit_df.to_string(index=False))


    def run_strategy(self):
        self.calculate_indicators()
        self.generate_signals()
        self.calculate_profit()

strategy = TradingStrategy(data)
strategy.run_strategy()


   Buy Price   Sell Price    Buy RSI   Sell RSI     Buy EPS    Sell EPS        Profit
 3960.821696  6424.189196  25.983035  19.267864  100.017033  118.650955   2463.367500
 3987.697387  6426.194316  23.339275  23.711089  100.590939  119.146636   2438.496929
 3986.168751  6435.839905  25.740084  26.697909  100.899883   119.08551   2449.671154
 4020.947909  6445.485493  33.038102  29.815766  100.663785  119.024383   2424.537584
 4007.458801  6455.131081  33.563551  33.073476  100.628552  118.963256   2447.672280
 3993.969693  6473.558922  34.105983  39.553788  100.593319    118.6589   2479.589229
 3980.480586   6439.57408  34.666237  23.481033  100.558086   119.03126   2459.093494
  3935.83235  6450.156272  37.493622  38.133741  100.975692   119.33282   2514.323922
 3945.806296  6494.505014  36.570229  67.099721  101.094883  119.153467   2548.698718
 3894.357438   6505.66379  37.272233  70.731189  101.372692  119.205685   2611.306351
 3916.564466  6516.822565  41.895644  74.163677  101.6

In [41]:
concatenated_df = pd.concat([strategy.data, strategy.signals], axis=0, ignore_index=True)

# Export the concatenated DataFrame to a CSV file
concatenated_df.to_csv('concatenated_data.csv', index=False)


### Trade Analysis Summary

I have used seasonality in data along with RSI to build the trading strategy

#### 1. Seasonality in Data

Upon analyzing the data, it is evident that there is a **seasonal pattern** in the stock market prices when observed across different years. The seasonality can be attributed to econonmy is dependent on agriculture and tourism. Esp, stock market value reaches it peak before October unlees RSI is already overbought. 

#### 2. Inverse Relationship Between RSI and Stock Market Value

Through the analysis of the **Relative Strength Index (RSI)** and its relationship with the stock market, I observed a **strong inverse correlation**. Specifically:
This inverse relationship between RSI and the stock market indicates that **RSI can be a useful indicator for timing market entries and exits**, as it helps identify conditions where the stock may be overbought or oversold.

#### 3. Percipitation and Forward EPS

In this analysis, two indicators—**Average Precipitation over the last month** and **Forward EPS**—could have been used to improve the projection of stock performance. These indicators serve as both **lagging** and **leading** indicators, respectively:

- **Average Precipitation**: A **lagging indicator**, reflecting historical weather conditions that could affect sectors like agriculture and tourism.
- **Forward EPS**: A **leading indicator**, used to project future earnings and profitability of stocks.

However, these indicators were not used correctly in the original strategy, and incorporating them properly could enhance the model’s forecasting ability.

#### Note
1 Trading strategy does not consider cost of executing the trade. If that is considered, we also have reduce the number of trade execution to avoid depleting the profit.
